In [26]:
from utils.objaverse_dataset import ObjaversePointCloudDataset

annotations_file = '/Users/kostyalbalint/Documents/Egyetem/7.Felev/Szakdolgozat/objaverse_labeling/concatenated_annotations.npy'
pc_dir = '/Users/kostyalbalint/Documents/Egyetem/7.Felev/Szakdolgozat/pointCloudsNoColor'

dataset = ObjaversePointCloudDataset(annotations_file=annotations_file, pc_dir=pc_dir, split='train', scale_mode='shape_unit', name_filter='table', load_to_mem=True) #4s 225ms

In [24]:
print(dataset.__len__())

31337


In [28]:
items = []
for i in range(dataset.__len__()):
    items.append(dataset.__getitem__(i)) # 25s 395ms

In [3]:
from utils.dataset import ShapeNetCore

dataset_path = './data/shapenet.hdf5'
categories = ['motorcycle']
scale_mode = 'shape_unit'

train_dset = ShapeNetCore(
    path=dataset_path,
    cates=categories,
    split='train',
    scale_mode=scale_mode,
)

In [4]:
train_dset.__len__()

286